<a href="https://colab.research.google.com/github/Octaxx/DLI-Assignment/blob/main/Model_Lead_Gavin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# === Step 1: Load DatasetCleaning.ipynb from GitHub ===
import requests, nbformat
from IPython import get_ipython

url = "https://raw.githubusercontent.com/Octaxx/DLI-Assignment/refs/heads/main/DatasetCleaning.ipynb"
response = requests.get(url)
notebook = nbformat.reads(response.text, as_version=4)
ipython = get_ipython()

print("⚙️ Running cells from DatasetCleaning.ipynb...\n")
for i, cell in enumerate(notebook.cells):
    if cell.cell_type == 'code':
        try:
            print(f"▶️ Executing cell {i+1}...")
            ipython.run_cell(cell.source)
        except Exception as e:
            print(f"❌ Error in cell {i+1}: {e}")
print("\n✅ All notebook cells executed.")

# Alias balanced dataset to df_G
df_G = df_balanced.copy()


⚙️ Running cells from DatasetCleaning.ipynb...

▶️ Executing cell 2...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
▶️ Executing cell 3...
📊 BEFORE BALANCING
--------------------------------------------------
Total rows before balancing: 18634
Class balance before balancing:
Email Type
Safe Email        11322
Phishing Email     7312
Name: count, dtype: int64


,Email Text,Email Type,Label
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,0
1,the other side of * galicismos * * galicismo *...,Safe Email,0
2,re : equistar deal tickets are you still avail...,Safe Email,0
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,1
4,software at incredibly low prices ( 86 % lower...,Phishing Email,1
5,global risk management operations sally congra...,Safe Email,0
6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email,0
7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email,1
8,"we owe you lots of money dear applicant , afte...",Phishing Email,1
9,re : coastal deal - with exxon participation u...,Safe Email,0


▶️ Executing cell 4...

📊 AFTER BALANCING (Oversampling)
--------------------------------------------------
Total rows after balancing: 22644
Class balance after balancing:
Email Type
Phishing Email    11322
Safe Email        11322
Name: count, dtype: int64
▶️ Executing cell 5...

🧾 SAMPLE OF CLEANED & BALANCED DATAFRAME
Total Rows        : 22644
Phishing Emails   : 11322
Safe Emails       : 11322

🧪 Cleaned & Balanced DataFrame (First 5 Rows):


,Email Text,Email Type,Label
0,INVESTMENT SCHOLARS CLUB- bringing you the lat...,Phishing Email,1
1,semantics : il dominio tempo-aspettuale il dom...,Safe Email,0
2,mature mom and her young horny lover ! . . woo...,Phishing Email,1
3,do you own a car ; starting december 7 th ford...,Phishing Email,1
4,rescue you from highprice medicaments and badp...,Phishing Email,1



🎯 Phishing Emails (First 5):


...


✅ Safe Emails (First 5):


,Email Text,Email Type,Label
1,semantics : il dominio tempo-aspettuale il dom...,Safe Email,0
5,http://www.bbc.co.uk/radio1/alt/nireland/ni_te...,Safe Email,0
6,Hi Damian.SuSe has a Sparc version I previousl...,Safe Email,0
7,"hpl nom for may 25 , 2001 ( see attached file ...",Safe Email,0
8,iatl 14 : final cfp the 14th annual meeting - ...,Safe Email,0


▶️ Executing cell 6...

🧠 Sample Extracted Features (First 5):


,Email Preview,char_count,word_count,exclamation_count,uppercase_ratio,has_link,has_login_word,has_html
0,INVESTMENT SCHOLARS CLUB- bringing you the lat...,3893,586,2,0.034,0,1,0
1,semantics : il dominio tempo-aspettuale il dom...,904,154,0,0.000,0,0,0
2,mature mom and her young horny lover ! . . woo...,664,141,1,0.000,0,0,0
3,do you own a car ; starting december 7 th ford...,792,170,0,0.000,0,0,0
4,rescue you from highprice medicaments and badp...,796,163,0,0.000,0,0,0



✅ Final Columns:
['Email Text', 'Email Type', 'Label', 'Email Preview', 'char_count', 'word_count', 'exclamation_count', 'uppercase_ratio', 'has_link', 'has_login_word', 'has_html']

✅ All notebook cells executed.


In [8]:
# === Step 2: Set Number of Training Runs ===
try:
    num_runs = int(input("Enter number of training runs: "))
except:
    num_runs = 3
    print("⚠️ Invalid input. Defaulting to 3 runs.")


Enter number of training runs: 100


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack, csr_matrix
import pandas as pd

# Step 3.1: Add Features
df_G['url_count'] = df_G['Email Text'].str.count(r'http[s]?://')
df_G['num_dots'] = df_G['Email Text'].str.count(r'\.')
df_G['has_attachment_word'] = df_G['Email Text'].str.contains(r'attachment|invoice|pdf', case=False).astype(int)
df_G['has_suspicious_word'] = df_G['Email Text'].str.contains(r'verify|account|confirm|login', case=False).astype(int)
df_G['avg_word_length'] = df_G['char_count'] / (df_G['word_count'] + 1)
df_G['special_char_count'] = df_G['Email Text'].str.count(r'[#$%^&*()]')
df_G['has_spam_phrase'] = df_G['Email Text'].str.contains(
    r'congratulations|you have won|limited time|click below', case=False
).astype(int)

# Step 3.2: TF-IDF (reduced for performance)
tfidf = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1, 1))
X_tfidf = tfidf.fit_transform(df_G['Email Text'])

# Step 3.3: Scaled Statistical Features
stat_features = df_G[[
    'char_count', 'word_count', 'exclamation_count', 'uppercase_ratio',
    'has_link', 'has_login_word', 'has_html', 'url_count', 'num_dots',
    'has_attachment_word', 'has_suspicious_word', 'avg_word_length',
    'special_char_count', 'has_spam_phrase'
]]
stat_scaled = MinMaxScaler().fit_transform(stat_features)

# Step 3.4: Semantic Cues
def extract_cues(text):
    text = text.lower()
    return pd.Series({
        'has_urgent': int(any(w in text for w in ['urgent', 'immediate', 'now'])),
        'has_threat': int(any(w in text for w in ['suspend', 'locked', 'compromised'])),
        'has_generic': int(any(w in text for w in ['dear user', 'dear customer', 'valued customer']))
    })

cue_features = df_G['Email Text'].apply(extract_cues)
cue_scaled = MinMaxScaler().fit_transform(cue_features)

# Step 3.5: Final Combined Features
X_combined = hstack([X_tfidf, csr_matrix(stat_scaled), csr_matrix(cue_scaled)])
y = df_G['Label']


In [10]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time

results = []

for run in range(num_runs):
    print(f"\n📦 Training Run {run + 1}/{num_runs}")
    start = time.time()

    X_train, X_test, y_train, y_test = train_test_split(
        X_combined, y, test_size=0.2, random_state=run, stratify=y
    )

    clf_rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
    clf_lr = LogisticRegression(max_iter=1000)
    clf_nb = MultinomialNB()
    clf_dt = DecisionTreeClassifier(max_depth=10, random_state=42)

    # ✅ Weighted Voting: Emphasize Random Forest
    ensemble = VotingClassifier(estimators=[
        ('rf', clf_rf),
        ('lr', clf_lr),
        ('nb', clf_nb),
        ('dt', clf_dt)
    ], voting='soft', weights=[3, 1, 1, 1])  # RF has more say

    ensemble.fit(X_train, y_train)
    y_pred = ensemble.predict(X_test)
    y_proba = ensemble.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)
    elapsed = time.time() - start

    print(f"✅ Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, ROC AUC: {roc:.4f}, Time: {elapsed:.2f}s")

    results.append({
        'Run': run + 1, 'Accuracy': acc, 'Precision': prec,
        'Recall': rec, 'F1': f1, 'ROC_AUC': roc, 'Time': elapsed
    })

# Results Summary
summary_df = pd.DataFrame(results)

try:
    import ace_tools as tools
    tools.display_dataframe_to_user(name="Model Training Results Summary", dataframe=summary_df)
except:
    print(summary_df)

print("\n📊 Averages:")
print(summary_df.mean(numeric_only=True).round(4))



📦 Training Run 1/100
✅ Accuracy: 0.9702, Precision: 0.9497, Recall: 0.9929, F1: 0.9708, ROC AUC: 0.9966, Time: 11.07s

📦 Training Run 2/100
✅ Accuracy: 0.9647, Precision: 0.9398, Recall: 0.9929, F1: 0.9657, ROC AUC: 0.9965, Time: 11.34s

📦 Training Run 3/100
✅ Accuracy: 0.9645, Precision: 0.9416, Recall: 0.9903, F1: 0.9653, ROC AUC: 0.9960, Time: 11.29s

📦 Training Run 4/100
✅ Accuracy: 0.9673, Precision: 0.9449, Recall: 0.9925, F1: 0.9681, ROC AUC: 0.9968, Time: 11.01s

📦 Training Run 5/100
✅ Accuracy: 0.9709, Precision: 0.9517, Recall: 0.9920, F1: 0.9715, ROC AUC: 0.9969, Time: 10.47s

📦 Training Run 6/100
✅ Accuracy: 0.9642, Precision: 0.9405, Recall: 0.9912, F1: 0.9652, ROC AUC: 0.9963, Time: 11.14s

📦 Training Run 7/100
✅ Accuracy: 0.9596, Precision: 0.9334, Recall: 0.9898, F1: 0.9608, ROC AUC: 0.9957, Time: 11.04s

📦 Training Run 8/100
✅ Accuracy: 0.9658, Precision: 0.9425, Recall: 0.9921, F1: 0.9667, ROC AUC: 0.9969, Time: 10.24s

📦 Training Run 9/100
✅ Accuracy: 0.9695, Precis